In [1]:
import numpy as np
import pandas as pd
import os
import pywikibot
import pickle
import torch
import json

/raid/home/kawinm/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
INDEX = "nifty500"

directory = "../../../../data/" + INDEX + "/"

ticker_list = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ticker, name = filename.split("-")

        df = pd.read_csv(f)

       # if df.shape[0] <= 2600: 
          #  continue 
        
        ticker_list[ticker] = [name]

print(len(ticker_list.keys()))
print(ticker_list.keys())

# Getting the WikiBase Id for the companies
# Some of the wanted tickers are not available.
wikicode_map = {}


# Obtained from Wikibase Query builder, using filter
# key 'part of' and value 's&p500
sp500_id = pd.read_csv("query.csv")

ticker_maps_mine = {}
for i in range(159):
    id = sp500_id.loc[i]['item'].split("/")[-1]
    #print(sp500_id.loc[i])
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item = pywikibot.ItemPage(repo, id)
    item_dict = item.get()
    try:
        d = item_dict['claims']['P414']
        ticker = d[0].qualifiers['P249'][0].toJSON()['datavalue']['value']
        print(ticker, id)
        ticker_maps_mine[ticker] = id
    except:
        print(sp500_id.loc[i], "Not available")

print(ticker_maps_mine)
# Even then, IDs for some tickers were not available, so I manually searched wikibase with company names
# and obtained the ID
my_map = {
    'MNST': 'Q1945295',
    'WBA': 'Q18712620',
    'EBAY': 'Q58024',
    'ENPH': 'Q17012234',
    'MTCH': 'Q23133268',
    'CHTR': 'Q2961234',
    'TMUS': 'Q3511885',
    'SIRI': 'Q3277465',
    'BKNG': 'Q18674747',
    'AMD': 'Q128896',
    'AVGO': 'Q4827416',
    'KDP': 'Q3116111',
    'SPLK': 'Q1835753',
    'CTRA': 'Q4035714',
    'PENN': 'Q7163201',
    'NEM': 'Q1785405',
    'GNRC': 'Q5531627',
    'TFC': 'Q795486',
    'PTC': 'Q1760641',
    'BKR': 'Q804353',
    'UAA': 'Q2031485',
    'SPGI': 'Q868587'    
}

ticker_id_map = {}
for ticker, val in ticker_list.items():
    if ticker in ticker_maps_mine:
        ticker_id_map[ticker] = ticker_maps_mine[ticker]
    elif ticker in my_map:
        ticker_id_map[ticker] = my_map[ticker]
    elif ticker in wikicode_map:
        ticker_id_map[ticker] = wikicode_map[ticker]
print(len(ticker_id_map.keys()))

with open('ticker_wiki_map.pkl', 'wb') as f:
    pickle.dump(ticker_id_map, f)

501
dict_keys(['RECLTD', 'MOTILALOFS', 'CHOLAHLDNG', 'ASTERDM', 'SHRIRAMFIN', 'APLLTD', 'HINDZINC', 'DLF', 'VGUARD', 'ANGELONE', 'JKCEMENT', 'JUBLFOOD', 'REDINGTON', 'EICHERMOT', '360ONE', 'KENNAMET', 'IBREALEST', 'MAHABANK', 'RBLBANK', 'AARTIIND', 'FACT', 'MCDOWELL_N', 'NTPC', 'FLUOROCHEM', 'BAJAJ_AUTO', 'IOB', 'RELIANCE', 'CANFINHOME', 'JBMA', 'KALYANKJIL', 'IRFC', 'TMB', 'TATAMOTORS', 'SWSOLAR', 'IEX', 'MCX', 'KIMS', 'BBTC', 'BLS', 'AMARAJABAT', 'MANAPPURAM', 'DEVYANI', 'EDELWEISS', 'EASEMYTRIP', 'SRF', 'SBILIFE', 'KALPATPOWR', 'LXCHEM', 'JYOTHYLAB', 'FDC', 'IDFCFIRSTB', 'HINDALCO', 'L&TFH', 'ICICIPRULI', '3MINDIA', 'JSL', 'GREENPANEL', 'HIKAL', 'CANBK', 'TECHM', 'NHPC', 'DCMSHRIRAM', 'TV18BRDCST', 'DRREDDY', 'JINDWORLD', 'TCS', 'BALAMINES', 'PHOENIXLTD', 'RAJESHEXPO', 'MFL', 'BHARATFORG', 'BAJFINANCE', 'MHRIL', 'RAMCOCEM', 'LATENTVIEW', 'AEGISCHEM', 'ZFCVINDIA', 'KEC', 'POLYCAB', 'SBICARD', 'OBEROIRLTY', 'WELCORP', 'GLAXO', 'SUNDARMFIN', 'BALRAMCHIN', 'NATCOPHARM', 'BLUESTARCO', 'U

KeyboardInterrupt: 

In [3]:
with open('ticker_wiki_map.pkl', 'rb') as f:
    ticker_wiki_map = pickle.load(f)

In [8]:
inverse_ticker_wiki_map = {ticker_wiki_map[k] : k for k in ticker_wiki_map}

In [9]:
# Relations to consider
relation_map = {
    'P127': 'Owned By - FO',
    'P155': 'Follows - FO - immediately prior item in a series of which the subject is a part, preferably use as qualifier of P179 [if the subject has replaced the preceding item, e.g. political offices',
    'P156': 'Followed By - FO',
    'P355': 'Subsidiary - FO',
    'P749': 'Parent Organisation - FO',
    'P31': 'Instance of',
    'P366': 'Use',
    'P452': 'Industry', 
    'P1056': 'Product or Material Produced',
    'P112': 'Founded By',
    'P169': 'CEO',
    'P113': 'Airline Hub',
    'P114': 'Airline Alliance',
    'P121': "Item Operated",
    'P1830': 'Owner of - FO',
    'P3320': 'Board Member',
    'P166': 'Award received',
    'P199': 'Business Division',
    'P361': 'Part of',
    'P400': 'Platform',
    'P2770': 'Source of Income',
    'P463': 'Member of',
    'P306': 'Operating System',
    'P1344': 'Participant of'
}

In [39]:
wiki_relations = {}

with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [40]:
for relation, rel_label in relation_map.items():
    for ticker, wiki_id in ticker_wiki_map.items():
        if (ticker, relation) in wiki_relations:
            continue

        site = pywikibot.Site("wikidata", "wikidata")
        repo = site.data_repository()
        item_dict = pywikibot.ItemPage(repo, wiki_id).get()
        claims = item_dict['claims']

        try:
            value = claims[relation]
            for snaks in value:
                snak = snaks.toJSON()
                snak_value = snak['mainsnak']['datavalue']['value']['numeric-id']
                wiki_relations[(ticker, relation)] = wiki_relations.get((ticker, relation), []) + [snak_value]
            print(ticker, relation, rel_label, wiki_relations[(ticker, relation)])
        except:
            wiki_relations[(ticker, relation)] = None

SBILIFE P127 Owned By - FO [1340361]
LICI P127 Owned By - FO [2767140]
ALKEM P127 Owned By - FO [17090484]
HAL P127 Owned By - FO [2767140]
AWL P749 Parent Organisation - FO [107646415, 1786712]
ADANITRANS P749 Parent Organisation - FO [4680631]
ADANIGREEN P749 Parent Organisation - FO [4680631]
IEX P31 Instance of [783794, 891723]
SBILIFE P31 Instance of [4830453, 891723]
CDSL P31 Instance of [4830453, 190864]
AWL P31 Instance of [4830453, 783794]
ADANITRANS P31 Instance of [4830453, 891723]
NIACL P31 Instance of [4830453, 891723]
LICI P31 Instance of [4830453]
INDIGO P31 Instance of [46970, 891723]
DIXON P31 Instance of [783794, 891723]
ALKEM P31 Instance of [4830453, 891723]
HAL P31 Instance of [4830453, 6881511, 891723]
HDFCLIFE P31 Instance of [4830453, 891723]
DMART P31 Instance of [4830453, 891723]
ADANIGREEN P31 Instance of [4830453, 891723]
BDL P31 Instance of [4830453, 891723]
KPITTECH P31 Instance of [161726, 891723]
IEX P452 Industry [1326624]
SBILIFE P452 Industry [43183]


In [41]:
with open('wiki_relations_value.pkl', 'wb') as f:
    pickle.dump(wiki_relations, f)

In [4]:
with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [5]:
print(len(wiki_relations))

2136


In [6]:
relation_knowledge_graph = []

In [10]:
# First-order relation
for key, value in wiki_relations.items():
    if value is None:
        continue
    for ids in value:
        if 'Q'+str(ids) in inverse_ticker_wiki_map:
            print(key, inverse_ticker_wiki_map['Q'+str(ids)])
            relation_knowledge_graph.append((key[0], key[1], inverse_ticker_wiki_map['Q'+str(ids)]))

('ADANIENT', 'P355') AWL
('AWL', 'P749') ADANIENT


In [11]:
second_order_relation_types = ['P452_P452', 'P127_P127', 'P355_P355', 'P355_P199', 
                  'P112_P112', 'P112_P127', 'P112_P169', 'P169_P112', 'P169_P127', 'P169_P169', 'P127_P112',
                  'P127_P169', 'P31_P366', 'P127_P3320', 'P463_P463', 'P1056_P306', 'P1056_P1056',
                  'P1056_P452', 'P31_P452', 'P199_P355', 'P452_P2770', 'P1344_P1344',
                  'P169_P3320', 'P452_P31', 'P452_P1056', 'P306_P1056', 'P121_P121', 'P31_P1056', 'P166_P166',
                  'P2770_P452', 'P366_P31', 'P1056_P31', 'P400_P1056', 'P3320_P127', 'P3320_P169', 'P1056_P400',
                  'P114_P114', 'P121_P1056', 'P113_P113', 'P127_P749', 'P127_P355', 'P749_P127', 'P1830_P127', 
                  'P1830_P749', 'P355_P127', 'P155_P355', 'P155_P155', 'P355_P155', 'P127_P1830', 'P749_P1830',
                  'P1056_P121']
first_order_relation_types = ['P355', 'P155', 'P127', 'P156', 'P749', 'P1830']

together = ['P355', 'P155', 'P127', 'P156', 'P749', 'P1830', 
            'P452_P452', 'P127_P127', 'P355_P355', 'P355_P199', 
                  'P112_P112', 'P112_P127', 'P112_P169', 'P169_P112', 'P169_P127', 'P169_P169', 'P127_P112',
                  'P127_P169', 'P31_P366', 'P127_P3320', 'P463_P463', 'P1056_P306', 'P1056_P1056',
                  'P1056_P452', 'P31_P452', 'P199_P355', 'P452_P2770', 'P1344_P1344',
                  'P169_P3320', 'P452_P31', 'P452_P1056', 'P306_P1056', 'P121_P121', 'P31_P1056', 'P166_P166',
                  'P2770_P452', 'P366_P31', 'P1056_P31', 'P400_P1056', 'P3320_P127', 'P3320_P169', 'P1056_P400',
                  'P114_P114', 'P121_P1056', 'P113_P113', 'P127_P749', 'P127_P355', 'P749_P127', 'P1830_P127', 
                  'P1830_P749', 'P355_P127', 'P155_P355', 'P155_P155', 'P355_P155', 'P127_P1830', 'P749_P1830',
                  'P1056_P121'
]
print(len(second_order_relation_types))

51


In [12]:
for relation in second_order_relation_types:
    print(relation)
    r1, r2 = relation.split('_')
    for ticker, wiki_id in ticker_wiki_map.items():
        value1 = wiki_relations[(ticker, r1)]
        if value1 is None:
            continue
        for ticker2, wiki_id2 in ticker_wiki_map.items():
            value2 = wiki_relations[(ticker2, r2)]
            if value2 is None:
                continue
            if set(value1).intersection(set(value2)) and ticker != ticker2:
                if (ticker, relation, ticker2) in relation_knowledge_graph:
                    continue
                print(ticker, relation, ticker2)
                relation_knowledge_graph.append((ticker, relation, ticker2))

P452_P452
MOTILALOFS P452_P452 BAJFINANCE
MOTILALOFS P452_P452 IIFL
MOTILALOFS P452_P452 LICI
MOTILALOFS P452_P452 KOTAKBANK
MOTILALOFS P452_P452 HDFCLIFE
MOTILALOFS P452_P452 BANKBARODA
APLLTD P452_P452 AUROPHARMA
APLLTD P452_P452 TORNTPHARM
APLLTD P452_P452 CIPLA
APLLTD P452_P452 ALKEM
APLLTD P452_P452 LUPIN
APLLTD P452_P452 IPCALAB
HINDZINC P452_P452 VEDL
IEX P452_P452 ADANITRANS
SBILIFE P452_P452 NIACL
TECHM P452_P452 TCS
TECHM P452_P452 MPHASIS
TECHM P452_P452 SONATSOFTW
TECHM P452_P452 ZENSARTECH
TECHM P452_P452 KPITTECH
TCS P452_P452 TECHM
TCS P452_P452 MPHASIS
TCS P452_P452 SONATSOFTW
TCS P452_P452 ZENSARTECH
TCS P452_P452 KPITTECH
BAJFINANCE P452_P452 MOTILALOFS
BAJFINANCE P452_P452 IIFL
BAJFINANCE P452_P452 LICI
BAJFINANCE P452_P452 KOTAKBANK
BAJFINANCE P452_P452 HDFCLIFE
BAJFINANCE P452_P452 BANKBARODA
TVSMOTOR P452_P452 MARUTI
TVSMOTOR P452_P452 HEROMOTOCO
TVSMOTOR P452_P452 ASHOKLEY
IIFL P452_P452 MOTILALOFS
IIFL P452_P452 BAJFINANCE
IIFL P452_P452 LICI
IIFL P452_P452 KOTA

In [13]:
relation_stat = {}
for head, relation, tail in relation_knowledge_graph:
    relation_stat[relation] = relation_stat.get(relation, 0) + 1
print(relation_stat)

{'P355': 1, 'P749': 1, 'P452_P452': 146, 'P127_P127': 22, 'P112_P112': 6, 'P112_P127': 5, 'P127_P112': 5, 'P463_P463': 2, 'P1056_P1056': 32, 'P31_P452': 2, 'P452_P31': 2, 'P31_P1056': 1, 'P1056_P31': 1, 'P127_P749': 2, 'P749_P127': 2}


In [3]:
nifty_id = {}
INDEX = "nifty500" 

directory = "../../../../data/" + INDEX + "/"

ticker_list = {}
company_to_id = {}
company_id    = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ticker, name = filename.split("-")
        name = name[:-5]
        ticker_list[ticker] = name
        df = pd.read_csv(f)
        if df.shape[0] <= 2800:    # 13 years
            print("Skipping file: Less Training-Testing samples [{0} samples]".format(df.shape[0]))
            continue

        if ticker not in company_to_id:
            company_to_id[ticker] = company_id
            company_id += 1
nifty_id = company_to_id
print(nifty_id)

Skipping file: Less Training-Testing samples [1198 samples]
Skipping file: Less Training-Testing samples [2787 samples]
Skipping file: Less Training-Testing samples [558 samples]
Skipping file: Less Training-Testing samples [816 samples]
Skipping file: Less Training-Testing samples [837 samples]
Skipping file: Less Training-Testing samples [1566 samples]
Skipping file: Less Training-Testing samples [799 samples]
Skipping file: Less Training-Testing samples [438 samples]
Skipping file: Less Training-Testing samples [477 samples]
Skipping file: Less Training-Testing samples [74 samples]
Skipping file: Less Training-Testing samples [836 samples]
Skipping file: Less Training-Testing samples [1285 samples]
Skipping file: Less Training-Testing samples [2671 samples]
Skipping file: Less Training-Testing samples [377 samples]
Skipping file: Less Training-Testing samples [1619 samples]
Skipping file: Less Training-Testing samples [343 samples]
Skipping file: Less Training-Testing samples [443 s

In [17]:
len(nifty_id)

336

In [18]:
nifty_rkg = [[], [], []]
for head, relation, tail in relation_knowledge_graph:
    if head in nifty_id and tail in nifty_id:
        nifty_rkg[0].append(nifty_id[head])
        
        rel_pos = together.index(relation)
        nifty_rkg[1].append(rel_pos)

        nifty_rkg[2].append(nifty_id[tail])
nifty_rkg[0] = torch.tensor(nifty_rkg[0])
nifty_rkg[1] = torch.tensor(nifty_rkg[1])
nifty_rkg[2] = torch.tensor(nifty_rkg[2])

with open("nifty500_relations_kg.pkl", "wb") as f:
    pickle.dump({'kg': nifty_rkg}, f)

In [4]:
wiki_relations = {}

with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [20]:
print(relation_map)

{'P127': 'Owned By - FO', 'P155': 'Follows - FO - immediately prior item in a series of which the subject is a part, preferably use as qualifier of P179 [if the subject has replaced the preceding item, e.g. political offices', 'P156': 'Followed By - FO', 'P355': 'Subsidiary - FO', 'P749': 'Parent Organisation - FO', 'P31': 'Instance of', 'P366': 'Use', 'P452': 'Industry', 'P1056': 'Product or Material Produced', 'P112': 'Founded By', 'P169': 'CEO', 'P113': 'Airline Hub', 'P114': 'Airline Alliance', 'P121': 'Item Operated', 'P1830': 'Owner of - FO', 'P3320': 'Board Member', 'P166': 'Award received', 'P199': 'Business Division', 'P361': 'Part of', 'P400': 'Platform', 'P2770': 'Source of Income', 'P463': 'Member of', 'P306': 'Operating System', 'P1344': 'Participant of'}


In [5]:
#print(wiki_relations)
import torch

# Creating relation Graph
start_id = 500
nifty_graph_map = {}
for key, value in wiki_relations.items():
    if key[0] in nifty_id:
        nifty_graph_map[key[0]] = nifty_id[key[0]]
    else:
        continue
    if not value:
        continue
    for node in value:
        if node not in nifty_graph_map:
            nifty_graph_map[node] = start_id 
            start_id += 1
print(start_id)

nifty_graph = [[], []]
for key, value in wiki_relations.items():
    if key[0] in nifty_id:
        if not value:
            continue
        for node in value:
            nifty_graph[0].append(nifty_graph_map[key[0]])
            nifty_graph[1].append(nifty_graph_map[node])
nifty_graph = torch.tensor(nifty_graph)
print(nifty_graph.shape)

with open("../relation_graph.pkl", "rb") as f:
    a = pickle.load(f)

with open("relation_graph.pkl", "wb") as f:
    a['nifty'] = nifty_graph
    pickle.dump(a, f)

682
torch.Size([2, 375])


In [8]:
nifty_hypergraph = [[], []]
edge_num = 0
for key, value in wiki_relations.items():
    if key[0] in nifty_id:
        if not value:
            continue
        nifty_hypergraph[0].append(nifty_graph_map[key[0]])
        nifty_hypergraph[1].append(edge_num)
        for node in value:
            nifty_hypergraph[0].append(nifty_graph_map[node])
            nifty_hypergraph[1].append(edge_num)
        edge_num += 1
nifty_hypergraph = torch.tensor(nifty_hypergraph)
print(nifty_hypergraph.shape)

with open("../relation_hypergraph.pkl", "rb") as f:
    a = pickle.load(f)

with open("relation_hypergraph.pkl", "wb") as f:
    a['nifty'] = nifty_hypergraph
    pickle.dump(a, f)

torch.Size([2, 608])


In [21]:
wiki_id_to_node_name_map = {}
non_english_nodes = []

In [22]:
with open('wiki_id_to_node_name_map.pkl', 'rb') as f:
    wiki_id_to_node_name_map = pickle.load(f)['map']
    non_english_nodes = pickle.load(f)['non_english_nodes']

EOFError: Ran out of input

In [47]:
for k, v in wiki_relations.items():
    if v == None:
        continue
    ticker, relation = k
    for nodes_id in v:
        if "Q"+str(nodes_id) in wiki_id_to_node_name_map or nodes_id in non_english_nodes:
            continue
            
        site = pywikibot.Site("wikidata", "wikidata")
        repo = site.data_repository()
        item_dict = pywikibot.ItemPage(repo, "Q"+str(nodes_id)).get()
        claims = item_dict['claims']
        try:
            node_name = item_dict['labels'].toJSON()['en']['value']
        except:
            non_english_nodes.append(nodes_id)
        print(relation, node_name)
        if "Q"+str(nodes_id) not in wiki_id_to_node_name_map:
            wiki_id_to_node_name_map["Q"+str(nodes_id)] = node_name



P127 State Bank of India
P127 Samprada Singh
P749 Adani Enterprises
P749 Wilmar International
P31 central securities depository
P31 airline
P452 electric utility
P452 insurance
P452 food processing
P452 personal care product
P452 consumer electronics industry
P452 home appliance industry
P452 reverse logistics
P452 electronics manufacturing services
P452 aerospace
P452 defense
P452 retail
P452 arms industry
P1056 life insurance
P1056 electric power transmission
P1056 television
P1056 washing machine
P1056 smartphone
P1056 LED lamp
P1056 closed-circuit television camera
P1056 recessed light
P1056 airlifter
P1056 fighter
P1056 transport aircraft
P1056 rocket engine
P1056 CE-20
P1056 supersonic aircraft
P1056 Tejas
P1056 HF-24 Marut
P1056 Dornier 228
P1056 aircraft maintenance
P1056 jet engine
P1056 helicopter
P1056 Dhruv
P1056 light combat aircraft
P1056 HAL Rudra
P1056 attack helicopter
P1056 Su-30MKI
P1056 HAL HTT-40
P1056 trainer aircraft
P1056 supermarket
P1056 ballistic missile
P105

In [48]:
for ticker, wiki_id in ticker_wiki_map.items():
    print(wiki_id)
    if str(wiki_id) in wiki_id_to_node_name_map or wiki_id in non_english_nodes:
        continue
        
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item_dict = pywikibot.ItemPage(repo, str(wiki_id)).get()
    claims = item_dict['claims']
    try:
        node_name = item_dict['labels'].toJSON()['en']['value']
    except:
        non_english_nodes.append(wiki_id)
    print(node_name)
    if str(nodes_id) not in wiki_id_to_node_name_map:
        wiki_id_to_node_name_map[str(nodes_id)] = node_name

Q6917706
Motilal Oswal Group
Q28451082
Alembic Pharmaceuticals Ltd
Q5766843
Hindustan Zinc Limited
Q3011374
DLF Limited
Q908931
Reliance Industries Limited
Q6020258
Indian Energy Exchange
Q7388955
SBI Life Insurance Company Limited
Q1619261
Hindalco Industries
Q1131463
Tech Mahindra India
Q13227919
Tata Consultancy Services
Q4901117
Bharat Forge
Q28173355
Bajaj Finance
Q6328340
KEC International
Q1966372
Natco Pharma
Q85747894
Blue Star
Q2723075
Dabur
Q7880419
UltraTech Cement
Q5060704
Central Depository Services Limited
Q109979847
Q5377917
Engineers India Limited
Q3300084
TVS Motor Company
Q6019346
India Infoline
Q96401486
Relaxo
Q3764687
Mphasis
Q1619376
Hindustan Unilever
Q83472792
Adani Transmission
Q4848822
Bajaj Electricals
Q7560822
Sonata Software
Q116784682
Astral Limited
Q13118486
New India Assurance
Q2767170
Life Insurance Corporation of India
Q963718
Maruti Suzuki
Q906943
Grasim Industries
Q6031376
Cyient
Q52825
ABB Group
Q1789944
Madras Rubber Factory
Q4822431
Aurobindo Pha

In [49]:
with open('wiki_id_to_node_name_map.pkl', 'wb') as f:
    pickle.dump({'map': wiki_id_to_node_name_map, 'non_english_nodes': non_english_nodes}, f)

In [23]:
with open('wiki_id_to_node_name_map.pkl', 'rb') as f:
    id_map = pickle.load(f)
    wiki_id_to_node_name_map = id_map['map']
    non_english_nodes = id_map['non_english_nodes']

In [24]:
instance_map = {}
instance_id_to_name_map = {}

In [53]:
for nodes_id, v in wiki_id_to_node_name_map.items():
    if nodes_id in instance_map:
        continue
    
    print(nodes_id, v)
    if nodes_id[0] != 'Q':
        nodes_id = 'Q'+nodes_id
        
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item_dict = pywikibot.ItemPage(repo, nodes_id).get()
    claims = item_dict['claims']
    
    try:
        if 'P31' in claims:
            value = claims['P31']
            for snaks in value:
                snak = snaks.toJSON()
                snak_value = snak['mainsnak']['datavalue']['value']['numeric-id']
                instance_map[nodes_id] = instance_map.get(nodes_id, []) + [snak_value]

                if snak_value in instance_id_to_name_map:
                    continue

                site = pywikibot.Site("wikidata", "wikidata")
                repo = site.data_repository()
                item_dict = pywikibot.ItemPage(repo, "Q"+str(snak_value)).get()
                claims = item_dict['claims']

                try:
                    node_name = item_dict['labels'].toJSON()['en']['value']
                    instance_id_to_name_map[snak_value] = node_name
                except:
                    non_english_nodes.append(nodes_id)
                print(nodes_id, node_name)
        else:
            print(claims)
    except:
        print("problem")

Q298547 Mukesh Ambani
Q298547 human
Q331715 Tata Group
Q331715 conglomerate
Q331715 multinational corporation
Q880942 The Blackstone Group
Q880942 business
Q880942 enterprise
Q880942 public company
Q157062 Unilever
Q157062 dual-listed company
Q157062 trademark
Q181642 Suzuki
Q181642 automobile manufacturer
Q181642 motorcycle manufacturer
Q1671804 Investor AB
Q1671804 investment company
Q1671804 holding company
Q10447369 Cevian Capital
Q219635 BlackRock
Q219635 company
Q2767140 Government of India
Q2767140 political system
Q2767140 organization
Q2767140 central government
Q1503753 Vedanta Resources
Q2536590 W.D. & H.O. Wills
Q17078993 Network 18
Q17078993 media conglomerate
Q18643672 Jio
Q3417395 Jamnagar Refinery
Q3417395 oil refining
Q29123941 Reliance Industries (United States)
Q96418672 Jio Platforms
Q116736440 Indiawin Sports
Q116736440 subsidiary
Q2002891 Novelis
Q29123400 Bharat Forge (Germany)
Q29124039 ABB
Q29124043 ABB (United Kingdom)
Q30338853 ABB (United States)
Q30338854 A

In [54]:
len(instance_map.items())

188

In [55]:
with open('wiki_id_to_node_name_map.pkl', 'wb') as f:
    pickle.dump({'map': wiki_id_to_node_name_map, 'non_english_nodes': non_english_nodes,
                 'id_instance': instance_map, 'id_instance_name': instance_id_to_name_map}, f)

In [25]:
with open("wiki_id_to_node_name_map.pkl", 'rb') as f:
    pkl_file = pickle.load(f)
    wiki_id_to_node_name_map = pkl_file['map']
    non_english_nodes = pkl_file['non_english_nodes']
    instance_map = pkl_file['id_instance']
    instance_id_to_name_map = pkl_file['id_instance_name']

with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [32]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

uri = "neo4j+s://ef378426.databases.neo4j.io"
user = "neo4j"
password = "_6AluugfMr__UTW5tfW2f_lmaqyIzvttqMut4wOA7D8"
app = App(uri, user, password)

In [27]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_relation(self, id1, id2, category, relation, name2, relation_id):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_relation, id1, id2, category, relation, name2, relation_id)
            for row in result:
                print("Created node between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_relation(tx, id1, id2, category, relation, name2, relation_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        
        query = (
            
            "MATCH (a:Asset) WHERE a.ticker = $ticker1 "
            "RETURN a"
        )
        result = tx.run(query, ticker1=id1, ticker2=id2)

        if len(list(result)) == 0:
            print("Creating asset 1 ", id1)
            query = (
            "CREATE (p1:Asset { ticker: $ticker1 }) "
            )
            result = tx.run(query, ticker1=id1)

        query = (
            "MATCH (c:"+category+") WHERE c.name = $name AND c.id = $ticker2 "
            "RETURN c"
        )
        result = tx.run(query, ticker2=id2, category=category, name=name2)
        
        if len(list(result)) == 0:
            query = (
                "CREATE (c1:"+category+" { name: $name, id: $id2 }) "
            )
            result = tx.run(query, name=name2, id2=id2)

        query = (
            "MATCH (a:Asset), (c:"+category+") WHERE a.ticker = $ticker1 AND c.id = $id2 "
            "CREATE (a)-[:"+relation+" { id: $relation_id}]->(c) "
            "RETURN a, c"
        )
        result = tx.run(query, ticker1=id1, id2=id2, relation_id=relation_id)
        try:
            return [{"p1": row["a"]["ticker"], "p2": row["c"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(self._find_and_return_person, person_name)
            for row in result:
                print("Found person: {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Person) "
            "WHERE p.name = $person_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]

In [28]:
for key, val in instance_map.items():
    if key in inverse_ticker_wiki_map:
        instance_map[key] = [1]
        print(key, inverse_ticker_wiki_map[key])
instance_id_to_name_map[1] = 'Asset'

Q109979847 AWL
Q107646415 ADANIENT


In [29]:
relation_map['P113']

'Airline Hub'

In [42]:
flag = True
for key, val in wiki_relations.items():
    if val == None:
        continue
    ticker, relation = key
    #if ticker != 'AZN' and flag:
    #    continue
    #flag = False
    if relation == 'P127':
        continue
        for v in val:
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])
            
            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]

            if 'Asset' in instances:
                category = 'Asset'
            elif 'human' in instances:
                category = 'Person'
            else:
                category = 'Company'

            #app.create_relation(ticker, v, category, "OWNED_BY", wiki_id_to_node_name_map['Q'+str(v)], 'P127')
    if relation == 'P155':
        continue
        for v in val:
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])
            
            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]

            if 'human' in instances or 'village' in instances:
                continue
            else:
                category = 'Company'

            #app.create_relation(ticker, v, category, "FOLLOWS", wiki_id_to_node_name_map['Q'+str(v)], 'P155')
    if relation == 'P156':
        continue
        for v in val:
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])
            
            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]

            if 'Asset' in instances:
                category = 'Asset'
            else:
                category = 'Company'

            #app.create_relation(ticker, v, category, "FOLLOWED_BY", wiki_id_to_node_name_map['Q'+str(v)], 'P156')
    if relation == 'P355':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])
            
            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]

            category = 'Company'
            #app.create_relation(ticker, v, category, "SUBSIDIARY", wiki_id_to_node_name_map['Q'+str(v)], 'P355')
    if relation == 'P749':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]
            if 'Asset' in instances:
                category = 'Asset'
            else:
                category = 'Company'

            #app.create_relation(ticker, v, category, "PARENT_ORGANISATION", wiki_id_to_node_name_map['Q'+str(v)], 'P749')
    if relation == 'P31':
        continue
        # Not added - Instances of - Most of them are public companies
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]
    if relation == 'P452':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)]]
            category = 'Industry'

            #app.create_relation(ticker, v, category, "FIELD_OF_ACTION", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P1056':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]
            category = 'Product'

            #app.create_relation(ticker, v, category, "PRODUCT_PRODUCED", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P112':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Person'

            #app.create_relation(ticker, v, category, "FOUNDED_BY", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P169':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Person'

            #app.create_relation(ticker, v, category, "CEO", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P113':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Airport'

            #app.create_relation(ticker, v, category, "AIRLINE_HUB", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P114':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Alliance'

            #app.create_relation(ticker, v, category, "AIRLINE_ALLIANCE", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P121':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            if ticker == 'META':
                continue
            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Product'

            #app.create_relation(ticker, v, category, "ITEM_OPERATED", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P1830':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            if 'Asset' in instances:
                category = 'Asset'
            elif 'business' in instances:
                category = 'Company'
            else:
                category = 'Product'

            #app.create_relation(ticker, v, category, "OWNER_OF", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P3320':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Person'

            #app.create_relation(ticker, v, category, "BOARD_MEMBER", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P166':
        continue
        # Not added - Award received
    if relation == 'P199':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Company'
            #app.create_relation(ticker, v, category, "BUSINESS_DIVISION", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P361':
        continue
        # Not added - Part of - S&P 500, NASDAQ-100
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]
    if relation == 'P400':
        continue
        # Not added - ios, android, platform
    if relation == 'P2770':
        continue
        # Not added - Source of Income - service
    if relation == 'P463':
        continue
        for v in val:
            if 'Q'+str(v) not in instance_map:
                continue
            print(ticker, wiki_id_to_node_name_map['Q'+str(v)], ": ", [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map])

            instances = [instance_id_to_name_map[instn] for instn in instance_map['Q'+str(v)] if instn in instance_id_to_name_map]

            category = 'Alliance'
            #app.create_relation(ticker, v, category, "MEMBER_OF", wiki_id_to_node_name_map['Q'+str(v)], relation)
    if relation == 'P1344':
        continue
        # Not added - Part of - trade conflict, disease, 
        # May add

MOTILALOFS financial services :  ['economic activity']
Created node between: MOTILALOFS, financial services
APLLTD pharmaceutical industry :  ['industry']
Created node between: APLLTD, pharmaceutical industry
HINDZINC mining :  ['economic activity']
Created node between: HINDZINC, mining
DLF real estate economics :  ['industry', 'academic discipline']
Created node between: DLF, real estate economics
RELIANCE petroleum industry :  ['industry']
Created node between: RELIANCE, petroleum industry
HINDALCO aluminium industry :  ['industry']
Created node between: HINDALCO, aluminium industry
TECHM IT service management :  ['specialty', 'industry']
Created node between: TECHM, IT service management
TECHM information technology consulting :  ['industry']
Created node between: TECHM, information technology consulting
TCS IT service management :  ['specialty', 'industry']
Created node between: TCS, IT service management
TCS information technology consulting :  ['industry']
Created node between: 

In [138]:
relation_map['P306']

'Operating System'

In [134]:
len(wiki_relations.items())

11064

In [139]:
# list comprehension with if statement
rela = set()
for key, rel in wiki_relations.items():
    rela.add(key[1])
print(rela)
print([relation_map[rel] for rel in rela])

{'P306', 'P155', 'P400', 'P1830', 'P3320', 'P463', 'P113', 'P112', 'P169', 'P1344', 'P749', 'P166', 'P121', 'P452', 'P1056', 'P2770', 'P156', 'P114', 'P355', 'P366', 'P127', 'P361', 'P199', 'P31'}
['Operating System', 'Follows - FO - immediately prior item in a series of which the subject is a part, preferably use as qualifier of P179 [if the subject has replaced the preceding item, e.g. political offices', 'Platform', 'Owner of - FO', 'Board Member', 'Member of', 'Airline Hub', 'Founded By', 'CEO', 'Participant of', 'Parent Organisation - FO', 'Award received', 'Item Operated', 'Industry', 'Product or Material Produced', 'Source of Income', 'Followed By - FO', 'Airline Alliance', 'Subsidiary - FO', 'Use', 'Owned By - FO', 'Part of', 'Business Division', 'Instance of']


In [1]:
kg_file = pd.read_csv("neo4j_query_table_data_2023-3-26.csv")
kg_file.head()

NameError: name 'pd' is not defined

In [26]:
json_data = kg_file['data'][0]

In [27]:
node_map = {}
relation_map = {}
relation_label_id, rid = {}, 0
for data in json_data.split('\n'):
    d = json.loads(data)
    print(d)
    if d['type'] =='node':
        node_map[d['id']] = d
    else:
        if d['label'] not in relation_label_id:
            relation_label_id[d['label']] = rid
            rid += 1
        relation_map[d['id']] = [d['start']['id'], relation_label_id[d['label']], d['end']['id']]

{'type': 'node', 'id': '0', 'labels': ['Person'], 'properties': {'name': 'Tommy Millner', 'id': 114689399}}
{'type': 'node', 'id': '1', 'labels': ['Person'], 'properties': {'name': 'Claudia Munce', 'id': 114689485}}
{'type': 'node', 'id': '2', 'labels': ['Person'], 'properties': {'name': 'Richelle Parham', 'id': 27452297}}
{'type': 'node', 'id': '3', 'labels': ['Person'], 'properties': {'name': 'Steve Rendle', 'id': 114689617}}
{'type': 'node', 'id': '4', 'labels': ['Person'], 'properties': {'name': 'Eugene Woods', 'id': 114689672}}
{'type': 'node', 'id': '5', 'labels': ['Person'], 'properties': {'name': 'Rudy de Leon', 'id': 7377714}}
{'type': 'node', 'id': '6', 'labels': ['Person'], 'properties': {'name': 'Cecil D. Haney', 'id': 5056036}}
{'type': 'node', 'id': '7', 'labels': ['Person'], 'properties': {'name': 'Mark Malcolm', 'id': 114899342}}
{'type': 'node', 'id': '8', 'labels': ['Person'], 'properties': {'name': 'James Mattis', 'id': 267902}}
{'type': 'node', 'id': '9', 'labels': 

In [28]:
nasdaq_c_id = {'SBUX': 0, 'VRSK': 1, 'ADBE': 2, 'NVDA': 3, 'AMGN': 4, 'ISRG': 5, 'KLAC': 6, 'AAPL': 7, 'REGN': 8, 'BIDU': 9, 'QCOM': 10, 'XEL': 11, 'MNST': 12, 'CTSH': 13, 'EA': 14, 'WBA': 15, 'ADSK': 16, 'EBAY': 17, 'ILMN': 18, 'NTES': 19, 'SWKS': 20, 'ADP': 21, 'MU': 22, 'AMZN': 23, 'MCHP': 24, 'NXPI': 25, 'FTNT': 26, 'SGEN': 27, 'TXN': 28, 'DXCM': 29, 'AZN': 30, 'VRSN': 31, 'MTCH': 32, 'CHTR': 33, 'AMAT': 34, 'FISV': 35, 'TMUS': 36, 'MSFT': 37, 'CTAS': 38, 'IDXX': 39, 'TSLA': 40, 'MAR': 41, 'AEP': 42, 'NFLX': 43, 'CDNS': 44, 'ALGN': 45, 'MELI': 46, 'GILD': 47, 'SIRI': 48, 'EXC': 49, 'CSCO': 50, 'LULU': 51, 'GOOGL': 52, 'MRVL': 53, 'ASML': 54, 'BIIB': 55, 'GOOG': 56, 'BKNG': 57, 'FAST': 58, 'ORLY': 59, 'ROST': 60, 'VRTX': 61, 'PCAR': 62, 'CMCSA': 63, 'PEP': 64, 'ATVI': 65, 'AMD': 66, 'ANSS': 67, 'AVGO': 68, 'INTU': 69, 'SNPS': 70, 'COST': 71, 'DLTR': 72, 'HON': 73, 'INTC': 74, 'PAYX': 75, 'CSX': 76, 'LRCX': 77, 'ADI': 78, 'MDLZ': 79, 'KDP': 80, 'CPRT': 81, 'ODFL': 82}
sp500_c_id = {'APH': 0, 'PG': 1, 'OMC': 2, 'VRTX': 3, 'EXC': 4, 'KMX': 5, 'STE': 6, 'SPG': 7, 'GE': 8, 'TMO': 9, 'EA': 10, 'CTAS': 11, 'PNR': 12, 'EIX': 13, 'SCHW': 14, 'ADI': 15, 'DLTR': 16, 'EQR': 17, 'GIS': 18, 'TRMB': 19, 'FCX': 20, 'UPS': 21, 'FDX': 22, 'SYY': 23, 'COF': 24, 'LEN': 25, 'WELL': 26, 'HSIC': 27, 'BDX': 28, 'GOOGL': 29, 'BEN': 30, 'LLY': 31, 'VFC': 32, 'WFC': 33, 'WHR': 34, 'UNP': 35, 'FMC': 36, 'GPC': 37, 'ABT': 38, 'IVZ': 39, 'WAT': 40, 'WEC': 41, 'ROK': 42, 'TSLA': 43, 'BRO': 44, 'LNC': 45, 'AME': 46, 'TDY': 47, 'DAL': 48, 'CTRA': 49, 'ROP': 50, 'PENN': 51, 'PEP': 52, 'REGN': 53, 'UNH': 54, 'NFLX': 55, 'DISH': 56, 'DHI': 57, 'ORLY': 58, 'BBWI': 59, 'JPM': 60, 'NWL': 61, 'A': 62, 'PFG': 63, 'MCHP': 64, 'ORCL': 65, 'NEM': 66, 'HBI': 67, 'AMT': 68, 'CDNS': 69, 'RE': 70, 'ED': 71, 'TDG': 72, 'ARE': 73, 'XEL': 74, 'PHM': 75, 'PEG': 76, 'CME': 77, 'CTSH': 78, 'ES': 79, 'AES': 80, 'GRMN': 81, 'PNW': 82, 'FTNT': 83, 'MO': 84, 'ICE': 85, 'MRO': 86, 'PLD': 87, 'DOV': 88, 'BSX': 89, 'ISRG': 90, 'VLO': 91, 'PPL': 92, 'DFS': 93, 'DUK': 94, 'ADP': 95, 'NVDA': 96, 'MAS': 97, 'ZBH': 98, 'C': 99, 'LEG': 100, 'WU': 101, 'IEX': 102, 'NXPI': 103, 'ANSS': 104, 'ADBE': 105, 'LVS': 106, 'ILMN': 107, 'CRL': 108, 'EXPE': 109, 'IFF': 110, 'DVN': 111, 'CNC': 112, 'WRB': 113, 'VTRS': 114, 'CNP': 115, 'MS': 116, 'LNT': 117, 'ROST': 118, 'CAT': 119, 'AMZN': 120, 'GNRC': 121, 'NTAP': 122, 'KEY': 123, 'PWR': 124, 'PRU': 125, 'HES': 126, 'DD': 127, 'F': 128, 'SBUX': 129, 'MLM': 130, 'WST': 131, 'MPWR': 132, 'IRM': 133, 'NVR': 134, 'JNJ': 135, 'AKAM': 136, 'LKQ': 137, 'REG': 138, 'SEE': 139, 'FISV': 140, 'WYNN': 141, 'PVH': 142, 'DPZ': 143, 'QCOM': 144, 'ACN': 145, 'INTU': 146, 'HSY': 147, 'LUV': 148, 'MHK': 149, 'TJX': 150, 'GWW': 151, 'UHS': 152, 'MSI': 153, 'BK': 154, 'ROL': 155, 'AFL': 156, 'EXR': 157, 'SWKS': 158, 'LH': 159, 'NSC': 160, 'VTR': 161, 'GOOG': 162, 'MSFT': 163, 'BBY': 164, 'AEP': 165, 'XRAY': 166, 'GD': 167, 'CVX': 168, 'HST': 169, 'AVGO': 170, 'WMB': 171, 'SIVB': 172, 'HBAN': 173, 'MKC': 174, 'GM': 175, 'TGT': 176, 'AIZ': 177, 'MDLZ': 178, 'IT': 179, 'MCD': 180, 'IPGP': 181, 'EBAY': 182, 'BAX': 183, 'NKE': 184, 'MGM': 185, 'IBM': 186, 'BKNG': 187, 'CAH': 188, 'RF': 189, 'FRT': 190, 'VZ': 191, 'WM': 192, 'TFC': 193, 'AVB': 194, 'MOS': 195, 'TER': 196, 'CCL': 197, 'DXCM': 198, 'AMGN': 199, 'MKTX': 200, 'CMI': 201, 'AJG': 202, 'PSA': 203, 'MA': 204, 'PPG': 205, 'UAL': 206, 'XOM': 207, 'ALB': 208, 'EMR': 209, 'EQIX': 210, 'CI': 211, 'CE': 212, 'AWK': 213, 'TSN': 214, 'ESS': 215, 'FLT': 216, 'SNA': 217, 'AON': 218, 'LDOS': 219, 'MTCH': 220, 'AAL': 221, 'BWA': 222, 'TPR': 223, 'EMN': 224, 'SO': 225, 'ALGN': 226, 'DE': 227, 'PM': 228, 'DXC': 229, 'SHW': 230, 'CMCSA': 231, 'MAR': 232, 'BAC': 233, 'DRI': 234, 'ZION': 235, 'CMS': 236, 'PKG': 237, 'PKI': 238, 'NI': 239, 'ALL': 240, 'ATVI': 241, 'SJM': 242, 'AIG': 243, 'LYV': 244, 'GLW': 245, 'SYK': 246, 'DLR': 247, 'UDR': 248, 'AXP': 249, 'PH': 250, 'TT': 251, 'EVRG': 252, 'WY': 253, 'LRCX': 254, 'ETN': 255, 'BIIB': 256, 'NUE': 257, 'PGR': 258, 'LMT': 259, 'RJF': 260, 'ODFL': 261, 'V': 262, 'RSG': 263, 'COP': 264, 'JBHT': 265, 'HON': 266, 'PTC': 267, 'SLB': 268, 'HPQ': 269, 'GPN': 270, 'FFIV': 271, 'CHTR': 272, 'CMG': 273, 'BKR': 274, 'ZBRA': 275, 'CLX': 276, 'EXPD': 277, 'TSCO': 278, 'SRE': 279, 'AVY': 280, 'LYB': 281, 'PCAR': 282, 'RTX': 283, 'MPC': 284, 'APD': 285, 'EW': 286, 'PFE': 287, 'EOG': 288, 'L': 289, 'COO': 290, 'SNPS': 291, 'USB': 292, 'FITB': 293, 'DHR': 294, 'CINF': 295, 'TTWO': 296, 'MAA': 297, 'GL': 298, 'RMD': 299, 'CMA': 300, 'INTC': 301, 'HUM': 302, 'BA': 303, 'PNC': 304, 'ULTA': 305, 'GPS': 306, 'NTRS': 307, 'BR': 308, 'K': 309, 'TMUS': 310, 'CSX': 311, 'UAA': 312, 'AMD': 313, 'HRL': 314, 'KLAC': 315, 'FAST': 316, 'ALK': 317, 'MET': 318, 'AMAT': 319, 'MCK': 320, 'RL': 321, 'BLK': 322, 'CBRE': 323, 'CL': 324, 'WDC': 325, 'LHX': 326, 'KMI': 327, 'PEAK': 328, 'TFX': 329, 'CVS': 330, 'KR': 331, 'VMC': 332, 'POOL': 333, 'TAP': 334, 'VRSN': 335, 'LOW': 336, 'DIS': 337, 'HD': 338, 'AMP': 339, 'TXT': 340, 'MTD': 341, 'MDT': 342, 'COST': 343, 'FIS': 344, 'FRC': 345, 'MNST': 346, 'CCI': 347, 'KIM': 348, 'D': 349, 'NEE': 350, 'CBOE': 351, 'TROW': 352, 'IP': 353, 'EL': 354, 'SBAC': 355, 'JNPR': 356, 'GS': 357, 'HAS': 358, 'LUMN': 359, 'GILD': 360, 'PXD': 361, 'MMM': 362, 'HII': 363, 'SWK': 364, 'ATO': 365, 'JCI': 366, 'TYL': 367, 'CAG': 368, 'AAPL': 369, 'CF': 370, 'ECL': 371, 'CPRT': 372, 'CRM': 373, 'EFX': 374, 'ADSK': 375, 'YUM': 376, 'DG': 377, 'DTE': 378, 'PAYX': 379, 'DGX': 380, 'STT': 381, 'CB': 382, 'TECH': 383, 'SPGI': 384, 'WMT': 385, 'VNO': 386, 'URI': 387, 'XYL': 388, 'MCO': 389, 'JKHY': 390, 'KO': 391, 'ITW': 392, 'VRSK': 393, 'HIG': 394, 'FE': 395, 'NOC': 396, 'CHRW': 397, 'KMB': 398, 'HCA': 399, 'AEE': 400, 'HAL': 401, 'RHI': 402, 'HOLX': 403, 'MSCI': 404, 'CPB': 405, 'BMY': 406, 'T': 407, 'MU': 408, 'IDXX': 409, 'NRG': 410, 'J': 411, 'LIN': 412, 'WAB': 413, 'CSCO': 414, 'TEL': 415, 'OXY': 416, 'BXP': 417, 'AZO': 418, 'ABC': 419, 'APA': 420, 'OKE': 421, 'AAP': 422, 'CHD': 423, 'O': 424, 'INCY': 425, 'NDAQ': 426, 'TRV': 427, 'STX': 428, 'WBA': 429, 'RCL': 430, 'ADM': 431, 'ETR': 432, 'STZ': 433, 'MRK': 434, 'MMC': 435, 'MTB': 436, 'AOS': 437, 'IPG': 438, 'DVA': 439, 'TXN': 440, 'BIO': 441}

In [19]:
# Missing nodes
for i in range(5000):
    if str(i) not in node_map:
        print(i)

540
541
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
713
714
715
716
717
718
719
720
721
834
835
1099
1100
1386
1387
1397
1398
1399
1400
1401
1402
1459
1460
1558
1559
2798
2799
2972
2973
3212
3213
3757
3758
4093
4094
4191
4192
4403
4404


In [29]:
nasdaq_c_id_to_kg_id = {}
sp_c_id_to_kg_id = {}

for k, v in node_map.items():
    if v['labels'][0] == 'Asset':
        if 'ticker' not in v['properties']:
            print(k, v)
            continue
        if v['properties']['ticker'] in nasdaq_c_id:
            nasdaq_c_id_to_kg_id[nasdaq_c_id[v['properties']['ticker']]] = v['id']
        if v['properties']['ticker'] in sp500_c_id:
            sp_c_id_to_kg_id[sp500_c_id[v['properties']['ticker']]] = v['id']


1405 {'type': 'node', 'id': '1405', 'labels': ['Asset'], 'properties': {'name': 'BlackRock', 'id': 219635}}
1409 {'type': 'node', 'id': '1409', 'labels': ['Asset'], 'properties': {'name': 'The Coca-Cola Company', 'id': 3295867}}
1420 {'type': 'node', 'id': '1420', 'labels': ['Asset'], 'properties': {'name': 'State Street Corporation', 'id': 2037125}}
1424 {'type': 'node', 'id': '1424', 'labels': ['Asset'], 'properties': {'name': 'T. Rowe Price', 'id': 3511946}}
1427 {'type': 'node', 'id': '1427', 'labels': ['Asset'], 'properties': {'name': 'Morgan Stanley', 'id': 334204}}
1441 {'type': 'node', 'id': '1441', 'labels': ['Asset'], 'properties': {'name': 'Northern Trust', 'id': 970695}}
1450 {'type': 'node', 'id': '1450', 'labels': ['Asset'], 'properties': {'name': 'General Mills', 'id': 629998}}
1467 {'type': 'node', 'id': '1467', 'labels': ['Asset'], 'properties': {'name': 'JPMorgan Chase', 'id': 192314}}
1470 {'type': 'node', 'id': '1470', 'labels': ['Asset'], 'properties': {'name': 'Ge

In [30]:
for k, v in sp500_c_id.items():
    if v not in sp_c_id_to_kg_id:
        print(k, ticker_wiki_map[k])
    

PNR Q7165001
CBOE Q30635731
LUMN Q5063092


In [31]:
nasdaq_c_id_to_kg_id_list, sp_c_id_to_kg_id_list = [], []
missing_node_id = 578 # for pnr, cboe, lumn

for i in range(len(sp500_c_id)):
    if i in nasdaq_c_id_to_kg_id:
        nasdaq_c_id_to_kg_id_list.append(int(nasdaq_c_id_to_kg_id[i]))
    if i in sp_c_id_to_kg_id:
        sp_c_id_to_kg_id_list.append(int(sp_c_id_to_kg_id[i]))
    else:
        sp_c_id_to_kg_id_list.append(missing_node_id)
        missing_node_id += 1

nasdaq_c_id_to_kg_id_list = torch.Tensor(nasdaq_c_id_to_kg_id_list).long()
sp_c_id_to_kg_id_list = torch.Tensor(sp_c_id_to_kg_id_list).long()

In [32]:
len(sp_c_id_to_kg_id_list)

442

In [33]:
total_kg = [[], [], []]

for k, v in relation_map.items():
    total_kg[0].append(int(v[0]))
    total_kg[1].append(int(v[1]))
    total_kg[2].append(int(v[2]))

total_kg[0] = torch.Tensor(total_kg[0])
total_kg[1] = torch.Tensor(total_kg[1])
total_kg[2] = torch.Tensor(total_kg[2])

In [34]:
with open("entire_kg.pkl", "wb") as f:
    pickle.dump({'kg': total_kg, 'nasdaq_map':nasdaq_c_id_to_kg_id_list, 'sp_map': sp_c_id_to_kg_id_list}, f)